# Transformer

In [1]:
import os
import json
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import math

In [77]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
from google.colab import files

uploaded = files.upload()

Saving transformer_df.csv to transformer_df.csv


In [92]:
df = pd.read_csv("transformer_df.csv")
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Reviews,Summary
0,"['Кроссовки удобные, но быстро порвались.', 'Л...","Кроссовки удобные и лёгкие, но не очень долгов..."
1,"['Телефон быстрый, батарея держит долго.', 'По...",Телефон хороший за свою цену. Камера хорошая д...
2,"['Чайник красивый, но шумный.', 'Пахнет пласти...","Чайник быстро кипятит воду и выглядит стильно,..."
3,"['Качество норм, но после стирки ткань стала ж...","Футболка приятная к телу, но могут быть пробле..."
4,"['Телефон красивый, но корпус маркий – остаютс...",Телефон с хорошей камерой и производительность...


In [93]:
df.head()

,Reviews,Summary
0,"['Кроссовки удобные, но быстро порвались.', 'Л...","Кроссовки удобные и лёгкие, но не очень долгов..."
1,"['Телефон быстрый, батарея держит долго.', 'По...",Телефон хороший за свою цену. Камера хорошая д...
2,"['Чайник красивый, но шумный.', 'Пахнет пласти...","Чайник быстро кипятит воду и выглядит стильно,..."
3,"['Качество норм, но после стирки ткань стала ж...","Футболка приятная к телу, но могут быть пробле..."
4,"['Телефон красивый, но корпус маркий – остаютс...",Телефон с хорошей камерой и производительность...


In [94]:
df["Reviews"] = df["Reviews"].apply(lambda x: " ".join(eval(x)).lower())
df['Reviews'][0]

'кроссовки удобные, но быстро порвались. лёгкие, но скользкие на мокром асфальте. размер в размер, не натирают. через месяц начала отклеиваться подошва. отличные, ношу каждый день!'

## Приведение всех отзывов к одному предложению

In [95]:
df['Reviews'] = df['Reviews'].apply(lambda x: "".join(x.lower()))

In [96]:
df.head()

,Reviews,Summary
0,"кроссовки удобные, но быстро порвались. лёгкие...","Кроссовки удобные и лёгкие, но не очень долгов..."
1,"телефон быстрый, батарея держит долго. после о...",Телефон хороший за свою цену. Камера хорошая д...
2,"чайник красивый, но шумный. пахнет пластиком д...","Чайник быстро кипятит воду и выглядит стильно,..."
3,"качество норм, но после стирки ткань стала жёс...","Футболка приятная к телу, но могут быть пробле..."
4,"телефон красивый, но корпус маркий – остаются ...",Телефон с хорошей камерой и производительность...


In [97]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

def encode_text(text):
    return tokenizer.encode(text, truncation=True, padding='max_length', max_length=64)

def decode_tokens(tokens):
    return tokenizer.decode(tokens, skip_special_tokens=True)

df["Reviews"] = df["Reviews"].apply(encode_text)
df["Summary"] = df["Summary"].apply(encode_text)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [98]:
df["Summary"]

,Summary
0,"[3, 2, 10338, 5345, 5345, 16811, 23110, 3, 370..."
1,"[3, 2, 1757, 19229, 2, 25752, 3, 2, 2044, 1033..."
2,"[3, 2, 2533, 2, 14391, 6652, 3, 2, 10458, 1033..."
3,"[3, 2, 3700, 6725, 2, 17674, 12095, 3, 2, 1470..."
4,"[3, 2, 1757, 19229, 2, 25752, 12681, 3, 2, 204..."
...,...
654,"[3, 2, 19229, 2, 12095, 3, 2, 10458, 7948, 2, ..."
655,"[3, 2, 2533, 2, 14391, 6652, 3, 2, 10458, 8452..."
656,"[3, 2, 20447, 8452, 6725, 2, 25752, 3, 2, 1045..."
657,"[3, 2, 6609, 1757, 2, 8194, 2, 3, 20447, 2, 27..."


In [99]:
df["Reviews"]

,Reviews
0,"[3, 6652, 10338, 5345, 5345, 16811, 23110, 3, ..."
1,"[3, 14982, 19229, 2, 25752, 3, 2, 10458, 8452,..."
2,"[3, 2, 2533, 2, 14391, 6652, 3, 6652, 7948, 26..."
3,"[3, 12095, 2, 1757, 24832, 2044, 3, 9890, 8452..."
4,"[3, 14982, 19229, 2, 25752, 3, 6652, 7948, 267..."
...,...
654,"[3, 2, 19229, 2, 12095, 3, 2, 10458, 7948, 2, ..."
655,"[3, 2, 2533, 2, 14391, 6652, 3, 2, 10458, 1033..."
656,"[12681, 20447, 8452, 6725, 2, 25752, 3, 2, 104..."
657,"[3, 2, 6609, 1757, 2, 8194, 2, 3, 20447, 2, 27..."


In [100]:
df.head()

,Reviews,Summary
0,"[3, 6652, 10338, 5345, 5345, 16811, 23110, 3, ...","[3, 2, 10338, 5345, 5345, 16811, 23110, 3, 370..."
1,"[3, 14982, 19229, 2, 25752, 3, 2, 10458, 8452,...","[3, 2, 1757, 19229, 2, 25752, 3, 2, 2044, 1033..."
2,"[3, 2, 2533, 2, 14391, 6652, 3, 6652, 7948, 26...","[3, 2, 2533, 2, 14391, 6652, 3, 2, 10458, 1033..."
3,"[3, 12095, 2, 1757, 24832, 2044, 3, 9890, 8452...","[3, 2, 3700, 6725, 2, 17674, 12095, 3, 2, 1470..."
4,"[3, 14982, 19229, 2, 25752, 3, 6652, 7948, 267...","[3, 2, 1757, 19229, 2, 25752, 12681, 3, 2, 204..."


In [101]:
df.to_csv("tokenized_transformer.csv")

In [102]:
from google.colab import files

files.download("tokenized_transformer.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [105]:
len(df["Reviews"][0])

64

In [107]:
len(df['Summary'][0])

64

In [16]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


# Реализация нейронной сети

In [125]:
X = np.array(df['Reviews'].tolist())
y = np.array(df['Summary'].tolist())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [120]:
X_train.shape

(527, 64)

In [121]:
y_train.shape

(527, 64)

In [122]:
X_test.shape

(132, 64)

In [139]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def build_transformer_model(hp):

    seq_len = 64
    vocab_size = hp.Int('vocab_size', min_value=8000, max_value=12000, step=2000)
    embedding_dim = hp.Choice('embedding_dim', values=[64, 128])
    num_heads = hp.Choice('num_heads', values=[4, 8])
    num_blocks = hp.Int('num_blocks', 1, 2)
    hidden_units = hp.Choice('hidden_units', values=[128, 256])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)

    inputs = layers.Input(shape=(seq_len,))
    x = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)


    x = layers.Dense(hidden_units)(x)

    for _ in range(num_blocks):
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=hidden_units,
            dropout=dropout_rate
        )(x, x)
        attention_output = layers.Dropout(dropout_rate)(attention_output)
        x = layers.LayerNormalization()(x + attention_output)

        ffn_output = layers.Dense(hidden_units, activation='relu')(x)
        ffn_output = layers.Dropout(dropout_rate)(ffn_output)
        x = layers.LayerNormalization()(x + ffn_output)

    x = layers.Dense(vocab_size, activation='softmax')(x)

    model = Model(inputs, x)

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


texts = ['Example sentence one', 'Another example sentence']
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(texts)
X_train = tokenizer.texts_to_sequences(texts)
X_test = tokenizer.texts_to_sequences(texts)

X_train = pad_sequences(X_train, maxlen=64)
X_test = pad_sequences(X_test, maxlen=64)


tuner = kt.Hyperband(
    build_transformer_model,
    objective='val_loss',
    max_epochs=10,
    hyperband_iterations=2,
    directory='kt_results',
    project_name='transformer_tuning'
)

tuner.search(X_train, X_train, epochs=10, batch_size=64, validation_data=(X_test, X_test))

best_model = tuner.get_best_models(num_models=1)[0]

loss, accuracy = best_model.evaluate(X_test, X_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Trial 54 Complete [00h 01m 36s]
val_loss: 1.8464710712432861

Best val_loss So Far: 0.5978848934173584
Total elapsed time: 00h 52m 18s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 180 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.9531 - loss: 0.5979
Test Loss: 0.5979
Test Accuracy: 0.9531


In [21]:
best_model.save('transformer.h5')

In [22]:
from google.colab import files

files.download("transformer.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [145]:
def decode_tokens(tokens):
    # Обратный словарь: индекс -> слово
    index_word = tokenizer.index_word
    return ' '.join([index_word.get(token, '') for token in tokens if token != 0])

example_input = np.expand_dims(X_train[1], axis=0)
prediction = best_model.predict(example_input)

# Получаем наиболее вероятные токены на каждом шаге
predicted_token_ids = np.argmax(prediction, axis=-1)[0]

decoded_text = decode_tokens(predicted_token_ids)
print("📝 Decoded Prediction:", decoded_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
📝 Decoded Prediction: 


In [146]:
example_input

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 2]],
      dtype=int32)